In [1]:
import datetime
import os
import random
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy
import statsmodels.api as sm
import pylab as py

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.linear_model import LinearRegression

import math
import time
import datetime
from dython.nominal import associations
from dython.nominal import identify_nominal_columns

In [2]:
data = pd.read_csv("../../dataset/final_bike_sharing.csv")

In [3]:
cols = ['tripduration','gender','dist','hour','birthyear','usertype']

In [4]:
df = data[cols]

## Before Imputation the distributaion is 

In [5]:
# for col in  ['tripduration','gender','dist','hour','birthyear']:
#     np.random.seed(19680801)
#     plt.figure(figsize=(20,8))
#     mu = df[col].mean()
#     sigma = df[col].std()
#     x = df[col]
#     num_bins = 500
#     fig, ax = plt.subplots()
#     # the histogram of the data
#     n, bins, patches = ax.hist(x, num_bins, density=True)
#     # add a 'best fit' line
#     y = ((1 / (np.sqrt(2 * np.pi) * sigma)) *
#          np.exp(-0.5 * (1 / sigma * (bins - mu))**2))
#     ax.plot(bins, y, '--')
#     ax.set_xlabel(col)
#     ax.set_ylabel('Probability density')
#     ax.set_title(f'Histogram of IQ: {mu}, {sigma}')
#     plt.plot([mu,mu],[0, 2], lw=1.8, label="mean")
# #     plt.plot([sigma,sigma],[0, 2], lw=1.8, label="std")
# #     ax.set_ylim([0,2.5])
# #     ax.set_xlim([0, 6])
#     # Tweak spacing to prevent clipping of ylabel
#     fig.tight_layout()
#     plt.show()

In [6]:
# for col in df.columns:
#     norm_cdf = scipy.stats.norm.cdf(df[col]) # calculate the cdf - also discrete
#     # plot the cdf
#     sns.lineplot(x=df[col], y=norm_cdf)
#     plt.show()
#     sm.qqplot(df[col], line ='45')
#     py.show()


In [7]:
for col in  ['tripduration','dist','hour','birthyear']:
    mu = df[col].mean()
    sigma = df[col].std()
    print(f"The variable {col} has mean = {mu} and standard deviation ={sigma} ")
print(f"The variable usertype categorical type mode is ", df['usertype'].mode() )  
print("gender mode is : " , df['gender'].mode() )

The variable tripduration has mean = 525.9314525266259 and standard deviation =731.0970649533257 
The variable dist has mean = 0.7842291293398377 and standard deviation =0.5421989613484405 
The variable hour has mean = 13.633500453206436 and standard deviation =5.156725828688158 
The variable birthyear has mean = 1980.439108882846 and standard deviation =10.094119857977768 
The variable usertype categorical type mode is  0    Subscriber
Name: usertype, dtype: object
gender mode is :  0    male
Name: gender, dtype: object


In [8]:
def split_dataset(df):
    X = df.drop('tripduration', axis=1)
    y = df['tripduration']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print('Train', X_train.shape, y_train.shape)
    print('Test', X_test.shape, y_test.shape)
    return X_train, X_test, y_train, y_test

In [9]:
## get the sample size such as 5%, 10%,20% etc
def get_sample_size(df, perc):
    return round(perc/100 * df.shape[0])

def get_an_index(index):
    li = []
    for i in index:
        li.append(i)
    return li   

def ampute_nan_values(X_train,col_name, perc):
    random.seed(100)
    index = get_an_index(X_train.index)
    sample_size = get_sample_size(X_train, perc)
    selected_index = random.sample(index, sample_size)
    for i in selected_index:
        if col_name in X_train.columns:
            X_train.at[i,col_name]= np.NaN
        else:
            X_train.loc[i] = np.NaN
    return X_train

###====================####
def convert_to_dataframe(y_train):
    return pd.DataFrame(y_train)

def convert_to_series(y_train):
    return y_train.squeeze()

def ampute_nan_values_drop(X_train,y_train,col_name, perc):
    random.seed(100)
    index = get_an_index(X_train.index)
    sample_size = get_sample_size(X_train, perc)
    selected_index = random.sample(index, sample_size)
    y_train = convert_to_dataframe(y_train)
    for i in selected_index:
        if col_name in X_train.columns:
            X_train.at[i,col_name]= np.NaN
        else:
            X_train.loc[i] = np.NaN
        y_train.loc[i] = np.NaN
    y_train = convert_to_series(y_train)
    return X_train, y_train

In [10]:
def sample_imputation_X(var,X_train,X_test):
    # extract a random sample
    random_sample_train = X_train[var].dropna().sample(X_train[var].isnull().sum(), random_state=0)
    random_sample_test = X_test[var].dropna().sample(X_test[var].isnull().sum(), random_state=0)
     # re-index the randomly extracted sample
    random_sample_train.index = X_train[X_train[var].isnull()].index
    random_sample_test.index = X_test[X_test[var].isnull()].index

    # replace the NA
    X_train.loc[X_train[var].isnull(), var] = random_sample_train
    X_test.loc[X_test[var].isnull(), var] = random_sample_test
    return X_train, X_test

In [11]:
def one_hot_encoding(X_train,X_test):
    categorical_features=identify_nominal_columns(X_train)
    print(categorical_features)
    one_hot_encoded_data_train = pd.get_dummies(X_train, columns = categorical_features)
    one_hot_encoded_data_test = pd.get_dummies(X_test, columns = categorical_features)
    return one_hot_encoded_data_train,one_hot_encoded_data_test

In [12]:
def model_evaluation(X_train,y_train):
    print(datetime.datetime.now())
    model_lr = LinearRegression(fit_intercept=True).fit(X_train, y_train) 
    print(datetime.datetime.now())
    return model_lr

In [13]:
def ampute_each_variables(variable, method,X_train,y_train,X_test,y_test):
    if method == "remove":
        X_train , y_train = ampute_nan_values_drop(X_train,y_train,variable,perc=30)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("y_train Amputed value count of varaible ", y_train.isnull().sum())
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test , y_test = ampute_nan_values_drop(X_test,y_test,variable,perc=30)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("y_test Amputed value count of varaible ", y_test.isnull().sum())
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    else:
        X_train = ampute_nan_values(X_train,variable,perc=30)
        print('Train set after Amputation', X_train.shape)
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test = ampute_nan_values(X_test,variable,perc=30)
        print('Test set after Amputation', X_test.shape)
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    
    if method == "remove":
        print("Simulate Imputation for NA")
        X_train.dropna(axis=0, inplace=True)
        y_train.dropna(inplace=True)
        print('Train after NA value removal Imputation', X_train.shape, y_train.shape)
        X_test.dropna(axis=0, inplace=True)
        y_test.dropna(inplace=True)
        print('Test after NA value removal Imputation', X_test.shape, y_test.shape)
    if method == "mode":
        print("Simulate Imputation for Mode")
        X_train.fillna(X_train.mode().iloc[0], inplace=True)
        print('Train after Mode Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test.mode().iloc[0], inplace=True)
        print('Test after Mode Imputation', X_test.shape, y_test.shape)
    if method == "mean":
        print("Simulate Imputation for Mean")
        X_train.fillna(X_train.mean(), inplace=True)
        print('Train after Mean Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test.mean(), inplace=True)
        print('Test after Mean Imputation', X_test.shape, y_test.shape)
    if method == "sample":
        print("Simple Imputation using sampling")
        X_train, X_test = sample_imputation_X(variable, X_train, X_test)        
            
    return X_train , y_train, X_test , y_test

In [14]:
def visualize_dist(df, col):
    np.random.seed(19680801)
    plt.figure(figsize=(20,8))
    mu = df[col].mean()
    sigma = df[col].std()
    x = df[col]
    num_bins = 500
    fig, ax = plt.subplots()
    # the histogram of the data
    n, bins, patches = ax.hist(x, num_bins, density=True)
    # add a 'best fit' line
    y = ((1 / (np.sqrt(2 * np.pi) * sigma)) *
         np.exp(-0.5 * (1 / sigma * (bins - mu))**2))
    ax.plot(bins, y, '--')
    ax.set_xlabel(col)
    ax.set_ylabel('Probability density')
    ax.set_title(f'Histogram of IQ: {mu}, {sigma}')
    plt.plot([mu,mu],[0, 2], lw=1.8, label="mean")
    #     plt.plot([sigma,sigma],[0, 2], lw=1.8, label="std")
    #     ax.set_ylim([0,2.5])
    #     ax.set_xlim([0, 6])
    # Tweak spacing to prevent clipping of ylabel
    fig.tight_layout()
    plt.show()

In [15]:
features_for_amputation =['gender','dist','hour','birthyear','usertype']

## Calculate mean and std before imputation

In [16]:
X_train, X_test, y_train, y_test = split_dataset(df)

Train (282432, 5) (282432,)
Test (70608, 5) (70608,)


In [17]:
for col in  ['dist','hour','birthyear']:
    mu = X_train[col].mean()
    sigma = X_train[col].std()
    print(f"The variable {col} has mean = {mu} and standard deviation ={sigma} ")
print(f"The variable usertype has mode ",  X_train['usertype'].mode())
print(f"The variable gender categorical type mode is ", X_train['gender'].mode() )

The variable dist has mean = 0.7848814720776504 and standard deviation =0.54278813457282 
The variable hour has mean = 13.633274558123725 and standard deviation =5.155597162884377 
The variable birthyear has mean = 1980.450076478586 and standard deviation =10.101242979840942 
The variable usertype has mode  0    Subscriber
Name: usertype, dtype: object
The variable gender categorical type mode is  0    male
Name: gender, dtype: object


In [18]:
for col in ['dist','hour','birthyear']:
    mu = X_test[col].mean()
    sigma = X_test[col].std()
    print(f"The variable {col} has mean = {mu} and standard deviation ={sigma} ")
print(f"The variable usertype has mode ",  X_test['usertype'].mode())
print(f"The variable gender categorical type mode is ", X_test['gender'].mode() ) 

The variable dist has mean = 0.7816197583886165 and standard deviation =0.539831786399696 
The variable hour has mean = 13.634404033537276 and standard deviation =5.161274457822756 
The variable birthyear has mean = 1980.3952384998868 and standard deviation =10.065528836178935 
The variable usertype has mode  0    Subscriber
Name: usertype, dtype: object
The variable gender categorical type mode is  0    male
Name: gender, dtype: object


In [19]:
cat_gender = ['gender_female', 'gender_male','gender_unknown']
cat_usertype = ['usertype_Customer', 'usertype_Subscriber']

## Imputation for NA ( drop NA values)

In [20]:
model_metrics ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"remove",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    if(var == 'gender'):
        for v in cat_gender:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                           "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    elif(var == 'usertype'):
        for v in cat_usertype:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                          "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    else:
        mu_train = X_train[var].mean()
        sigma_train = X_train[var].std()
        mu_test = X_test[var].mean()
        sigma_test = X_test[var].std()
        model_metrics[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                              "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                               "intercept":model_reg.intercept_,
                              "Mu_train":mu_train, "sigma_train":sigma_train,
                              "Mu_test":mu_test, "sigma_test":sigma_test}
    
    del X_train
    del X_test
    del y_train
    del y_test

Now running model is .... gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5) (282432,)
y_train Amputed value count of varaible  84730
X_train Amputed value count of varaible  84730
Test set after Amputation (70608, 5) (70608,)
y_test Amputed value count of varaible  21182
X_test Amputed value count of varaible  21182
Simulate Imputation for NA
Train after NA value removal Imputation (197702, 5) (197702,)
Test after NA value removal Imputation (49426, 5) (49426,)
['gender', 'usertype']
2023-02-19 15:14:07.840329
2023-02-19 15:14:07.974071
Now running model is .... dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5) (282432,)
y_train Amputed value count of varaible  84730
X_train Amputed value count of varaible  84730
Test set after Amputation (70608, 5) (70608,)
y_test Amputed value count of varaible  21182
X_test Amputed value count of varaible  21182
Simulate Imputation for NA
Train after NA v

In [21]:
model_metrics

{'gender_female': {'MAE': 279.21711401617745,
  'MSE': 420130.14623705694,
  'RMSE': 648.1744720652433,
  'R2': 0.17175428879313015,
  'intercept': 4139.21243948069,
  'Mu_train': 0.21241565588613165,
  'sigma_train': 0.4090184485129352,
  'Mu_test': 0.21213531339780684,
  'sigma_test': 0.40882429450055874},
 'gender_male': {'MAE': 279.21711401617745,
  'MSE': 420130.14623705694,
  'RMSE': 648.1744720652433,
  'R2': 0.17175428879313015,
  'intercept': 4139.21243948069,
  'Mu_train': 0.7274787306147636,
  'sigma_train': 0.44525771179413776,
  'Mu_test': 0.7282604297333387,
  'sigma_test': 0.44486085488313537},
 'gender_unknown': {'MAE': 279.21711401617745,
  'MSE': 420130.14623705694,
  'RMSE': 648.1744720652433,
  'R2': 0.17175428879313015,
  'intercept': 4139.21243948069,
  'Mu_train': 0.06010561349910471,
  'sigma_train': 0.23768301259099936,
  'Mu_test': 0.05960425686885445,
  'sigma_test': 0.23675456385379678},
 'dist': {'MAE': 279.21711401617745,
  'MSE': 420130.14623705694,
  'RM

### Bootstrap with drop

In [22]:
bootstrap_data_drop ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"remove",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx = np.arange(y_train.shape[0])
    bootstarp_train_accuracies = {}
    for i in range(501):
        train_idx = rng.choice(idx, size=idx.shape[0], replace=True)
        test_idx = np.setdiff1d(idx, train_idx, assume_unique=False)
        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]
        boot_test_X, boot_test_y = X_train.iloc[test_idx], y_train.iloc[test_idx]
        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_drop[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5) (282432,)
y_train Amputed value count of varaible  84730
X_train Amputed value count of varaible  84730
Test set after Amputation (70608, 5) (70608,)
y_test Amputed value count of varaible  21182
X_test Amputed value count of varaible  21182
Simulate Imputation for NA
Train after NA value removal Imputation (197702, 5) (197702,)
Test after NA value removal Imputation (49426, 5) (49426,)
['gender', 'usertype']
Now running model is .... dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5) (282432,)
y_train Amputed value count of varaible  84730
X_train Amputed value count of varaible  84730
Test set after Amputation (70608, 5) (70608,)
y_test Amputed value count of varaible  21182
X_test Amputed value count of varaible  21182
Simulate Imputation for NA
Train after NA value removal Imputation (197702, 5) (197702,)
Test aft

In [23]:
bootstrap_data_drop

{'gender': {'R2': [0.17027847551248348, 0.18588588915925847],
  'RMSE': [651.1930436503169, 687.4213614773985],
  'intercept': [3442.1923463760604, 4858.746632123913]},
 'dist': {'R2': [0.17027847551248348, 0.18588588915925847],
  'RMSE': [651.1930436503169, 687.4213614773985],
  'intercept': [3442.1923463760604, 4858.746632123913]},
 'hour': {'R2': [0.17027847551248348, 0.18588588915925847],
  'RMSE': [651.1930436503169, 687.4213614773985],
  'intercept': [3442.1923463760604, 4858.746632123913]},
 'birthyear': {'R2': [0.17027847551248348, 0.18588588915925847],
  'RMSE': [651.1930436503169, 687.4213614773985],
  'intercept': [3442.1923463760604, 4858.746632123913]},
 'usertype': {'R2': [0.17027847551248348, 0.18588588915925847],
  'RMSE': [651.1930436503169, 687.4213614773985],
  'intercept': [3442.1923463760604, 4858.746632123913]}}

### Mode Replacement

In [24]:
model_metrics_mode ={}
for var in features_for_amputation:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mode",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    if(var == 'gender'):
        for v in cat_gender:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics_mode[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                           "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    elif(var == 'usertype'):
        for v in cat_usertype:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics_mode[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                            "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    else:
        mu_train = X_train[var].mean()
        sigma_train = X_train[var].std()
        mu_test = X_test[var].mean()
        sigma_test = X_test[var].std()
        model_metrics_mode[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                              "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                                "intercept":model_reg.intercept_,
                              "Mu_train":mu_train, "sigma_train":sigma_train,
                              "Mu_test":mu_test, "sigma_test":sigma_test}
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  84730
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  21182
Simulate Imputation for Mode
Train after Mode Imputation (282432, 5) (282432,)
Test after Mode Imputation (70608, 5) (70608,)
['gender', 'usertype']
2023-02-19 15:19:18.103502
2023-02-19 15:19:18.188060
Now model is running for varaible ==== dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  84730
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  21182
Simulate Imputation for Mode
Train after Mode Imputation (282432, 5) (282432,)
Test after Mode Imputation (70608, 5) (70608,)
['gender', 'usertype']
2023-02-19 15:19:20.176240
2023-02-19 15:19:20.273324
Now model is running for varaible ==== hour
Train (282432, 5

In [25]:
model_metrics_mode

{'gender_female': {'MAE': 278.8692481491538,
  'MSE': 429416.4352502737,
  'RMSE': 655.2987374093389,
  'R2': 0.16690482705155496,
  'intercept': 4572.206099409003,
  'Mu_train': 0.14869065828234762,
  'sigma_train': 0.3557839156122109,
  'Mu_test': 0.1484959211420802,
  'sigma_test': 0.3555934101929993},
 'gender_male': {'MAE': 278.8692481491538,
  'MSE': 429416.4352502737,
  'RMSE': 655.2987374093389,
  'R2': 0.16690482705155496,
  'intercept': 4572.206099409003,
  'Mu_train': 0.809235497394063,
  'sigma_train': 0.3929045096960431,
  'Mu_test': 0.8097807613868117,
  'sigma_test': 0.3924768292136893},
 'gender_unknown': {'MAE': 278.8692481491538,
  'MSE': 429416.4352502737,
  'RMSE': 655.2987374093389,
  'R2': 0.16690482705155496,
  'intercept': 4572.206099409003,
  'Mu_train': 0.0420738443235894,
  'sigma_train': 0.2007580101764719,
  'Mu_test': 0.04172331747110809,
  'sigma_test': 0.19995761680485905},
 'dist': {'MAE': 292.20408652186177,
  'MSE': 433809.2918253957,
  'RMSE': 658.64

### Bootstrap with mode

In [26]:
bootstrap_data_mode = {}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mode",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx = np.arange(y_train.shape[0])
    bootstarp_train_accuracies = {}
    for i in range(501):
        train_idx = rng.choice(idx, size=idx.shape[0], replace=True)
        test_idx = np.setdiff1d(idx, train_idx, assume_unique=False)
        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]
        boot_test_X, boot_test_y = X_train.iloc[test_idx], y_train.iloc[test_idx]
        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_mode[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  84730
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  21182
Simulate Imputation for Mode
Train after Mode Imputation (282432, 5) (282432,)
Test after Mode Imputation (70608, 5) (70608,)
['gender', 'usertype']
Now running model is .... dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  84730
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  21182
Simulate Imputation for Mode
Train after Mode Imputation (282432, 5) (282432,)
Test after Mode Imputation (70608, 5) (70608,)
['gender', 'usertype']
Now running model is .... hour
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  84730
Test set after Amputatio

In [27]:
bootstrap_data_mode

{'gender': {'R2': [0.16948671457369874, 0.18315528677655357],
  'RMSE': [650.039544027035, 682.3413824741604],
  'intercept': [3952.6025169830027, 5193.349654425764]},
 'dist': {'R2': [0.16215710650658666, 0.17513979523209539],
  'RMSE': [653.419984255818, 685.3838738129667],
  'intercept': [3271.22334004212, 4395.419361195757]},
 'hour': {'R2': [0.16971061636973578, 0.18318692609141957],
  'RMSE': [650.0375570143433, 682.3338056689058],
  'intercept': [3683.4251399814348, 4813.615340846018]},
 'birthyear': {'R2': [0.16999827843669152, 0.18343939659903696],
  'RMSE': [649.8421910291186, 682.2337119731031],
  'intercept': [2483.4696082320625, 3598.298809583811]},
 'usertype': {'R2': [0.13695173300382452, 0.1508715616000464],
  'RMSE': [663.0242692664726, 696.4180504500068],
  'intercept': [2904.456667780773, 4121.242701706049]}}

## Mean Imputation

In [28]:
variables =['dist','hour','birthyear']
model_metrics_mean ={}
for var in variables:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mean",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    model_metrics_mean[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                            "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  84730
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  21182
Simulate Imputation for Mean


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_2668\3519124876.py:35: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_train.fillna(X_train.mean(), inplace=True)


Train after Mean Imputation (282432, 5) (282432,)


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_2668\3519124876.py:37: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_test.fillna(X_test.mean(), inplace=True)


Test after Mean Imputation (70608, 5) (70608,)
['gender', 'usertype']
2023-02-19 15:29:12.828394
2023-02-19 15:29:12.902478
Now model is running for varaible ==== hour
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  84730
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  21182
Simulate Imputation for Mean


...

C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_2668\3519124876.py:37: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_test.fillna(X_test.mean(), inplace=True)


Test after Mean Imputation (70608, 5) (70608,)
['gender', 'usertype']
2023-02-19 15:32:33.766315
2023-02-19 15:32:33.842862
Now model is running for varaible ==== birthyear
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  84730
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  21182
Simulate Imputation for Mean


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_2668\3519124876.py:35: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_train.fillna(X_train.mean(), inplace=True)


Train after Mean Imputation (282432, 5) (282432,)
Test after Mean Imputation (70608, 5) (70608,)
['gender', 'usertype']
2023-02-19 15:35:49.690219
2023-02-19 15:35:49.777736


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_2668\3519124876.py:37: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_test.fillna(X_test.mean(), inplace=True)


In [29]:
model_metrics_mean

{'dist': {'MAE': 291.1887971476297,
  'MSE': 433644.9919233972,
  'RMSE': 658.5172677488398,
  'R2': 0.1587011583893042,
  'intercept': 3837.1015651514567,
  'Mu_train': 0.9563824212553819,
  'sigma_train': 0.2042430739809072,
  'Mu_test': 0.9570020394289599,
  'sigma_test': 0.20285393451678185},
 'hour': {'MAE': 278.8191117599752,
  'MSE': 429153.65113467735,
  'RMSE': 655.0981996118425,
  'R2': 0.1674146449352213,
  'intercept': 4229.5175783072755,
  'Mu_train': 0.9563824212553819,
  'sigma_train': 0.2042430739809072,
  'Mu_test': 0.9570020394289599,
  'sigma_test': 0.20285393451678185},
 'birthyear': {'MAE': 278.747867226137,
  'MSE': 429141.59773600876,
  'RMSE': 655.0889998588045,
  'R2': 0.16743802929460794,
  'intercept': 4100.280860668483,
  'Mu_train': 0.9563824212553819,
  'sigma_train': 0.2042430739809072,
  'Mu_test': 0.9570020394289599,
  'sigma_test': 0.20285393451678185}}

### Bootstrap with mean 

In [30]:
bootstrap_data_mean ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mean",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx = np.arange(y_train.shape[0])
    bootstarp_train_accuracies = {}
    for i in range(501):
        train_idx = rng.choice(idx, size=idx.shape[0], replace=True)
        test_idx = np.setdiff1d(idx, train_idx, assume_unique=False)
        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]
        boot_test_X, boot_test_y = X_train.iloc[test_idx], y_train.iloc[test_idx]
        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_mean[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  84730
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  21182
Simulate Imputation for Mean


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_2668\3519124876.py:35: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_train.fillna(X_train.mean(), inplace=True)


Train after Mean Imputation (282432, 5) (282432,)


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_2668\3519124876.py:37: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_test.fillna(X_test.mean(), inplace=True)


Test after Mean Imputation (70608, 5) (70608,)
['gender', 'usertype']
Now running model is .... dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  84730
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  21182
Simulate Imputation for Mean


...

Test after Mean Imputation (70608, 5) (70608,)
['gender', 'usertype']
Now running model is .... usertype
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  84730
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  21182
Simulate Imputation for Mean


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_2668\3519124876.py:35: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_train.fillna(X_train.mean(), inplace=True)


Train after Mean Imputation (282432, 5) (282432,)


C:\Users\Hinakoushar\AppData\Local\Temp\ipykernel_2668\3519124876.py:37: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  X_test.fillna(X_test.mean(), inplace=True)


Test after Mean Imputation (70608, 5) (70608,)
['gender', 'usertype']


In [31]:
bootstrap_data_mean

{'gender': {'R2': [0.1696499996565437, 0.18321673414152612],
  'RMSE': [649.9773680175595, 682.304614110603],
  'intercept': [3886.4350570184297, 5132.057118764554]},
 'dist': {'R2': [0.16249555975340202, 0.17533773570603622],
  'RMSE': [653.2986594976559, 685.2764362197564],
  'intercept': [3280.495591019094, 4399.658763647251]},
 'hour': {'R2': [0.16993946940949106, 0.18329487707226272],
  'RMSE': [649.9145406243812, 682.2561031947929],
  'intercept': [3683.142429779408, 4817.313096140191]},
 'birthyear': {'R2': [0.17008785166170587, 0.18360839943341656],
  'RMSE': [649.7942684570487, 682.1731350222277],
  'intercept': [3463.830077076648, 4827.239023228878]},
 'usertype': {'R2': [0.1384267532238745, 0.15209264223389451],
  'RMSE': [662.5476553284448, 695.8474731135385],
  'intercept': [2512.772205881981, 3712.0955080118965]}}

## Sampling Replacement Imputation

In [32]:
model_metrics_sample ={}
for var in features_for_amputation:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"sample",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    if(var == 'gender'):
        for v in cat_gender:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics_sample[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                            "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    elif(var == 'usertype'):
        for v in cat_usertype:
            mu_train = X_train[v].mean()
            sigma_train = X_train[v].std()
            mu_test = X_test[v].mean()
            sigma_test = X_test[v].std()
            model_metrics_sample[v] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                           "intercept":model_reg.intercept_,
                          "Mu_train":mu_train, "sigma_train":sigma_train,
                          "Mu_test":mu_test, "sigma_test":sigma_test}
    else:
        mu_train = X_train[var].mean()
        sigma_train = X_train[var].std()
        mu_test = X_test[var].mean()
        sigma_test = X_test[var].std()
        model_metrics_sample[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                              "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred),
                               "intercept":model_reg.intercept_,
                              "Mu_train":mu_train, "sigma_train":sigma_train,
                              "Mu_test":mu_test, "sigma_test":sigma_test}
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  84730
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  21182
Simple Imputation using sampling
['gender', 'usertype']
2023-02-19 15:55:18.717874
2023-02-19 15:55:18.804556
Now model is running for varaible ==== dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  84730
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  21182
Simple Imputation using sampling
['gender', 'usertype']
2023-02-19 15:55:20.574150
2023-02-19 15:55:20.641146
Now model is running for varaible ==== hour
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  84730
Test set after Amputation (70608, 5)
X_test Amputed value count

In [33]:
model_metrics_sample

{'gender_female': {'MAE': 279.0582596954936,
  'MSE': 429574.5118342138,
  'RMSE': 655.4193404487037,
  'R2': 0.1665981484332577,
  'intercept': 4729.485666497717,
  'Mu_train': 0.21225286086562428,
  'sigma_train': 0.40890362670022007,
  'Mu_test': 0.21242635395422615,
  'sigma_test': 0.4090278322720775},
 'gender_male': {'MAE': 279.0582596954936,
  'MSE': 429574.5118342138,
  'RMSE': 655.4193404487037,
  'R2': 0.1665981484332577,
  'intercept': 4729.485666497717,
  'Mu_train': 0.7278991049172898,
  'sigma_train': 0.4450423566960799,
  'Mu_test': 0.7278495354634036,
  'sigma_test': 0.4450701008174726},
 'gender_unknown': {'MAE': 279.0582596954936,
  'MSE': 429574.5118342138,
  'RMSE': 655.4193404487037,
  'R2': 0.1665981484332577,
  'intercept': 4729.485666497717,
  'Mu_train': 0.059848034217085885,
  'sigma_train': 0.23720549369407865,
  'Mu_test': 0.05972411058237027,
  'sigma_test': 0.2369766582294243},
 'dist': {'MAE': 299.59747846998687,
  'MSE': 436935.00529423996,
  'RMSE': 661

### Bootstrap with sample

In [34]:
bootstrap_data_sample ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"sample",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx = np.arange(y_train.shape[0])
    bootstarp_train_accuracies = {}
    for i in range(501):
        train_idx = rng.choice(idx, size=idx.shape[0], replace=True)
        test_idx = np.setdiff1d(idx, train_idx, assume_unique=False)
        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]
        boot_test_X, boot_test_y = X_train.iloc[test_idx], y_train.iloc[test_idx]
        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_sample[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... gender
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  84730
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  21182
Simple Imputation using sampling
['gender', 'usertype']
Now running model is .... dist
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  84730
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  21182
Simple Imputation using sampling
['gender', 'usertype']
Now running model is .... hour
Train (282432, 5) (282432,)
Test (70608, 5) (70608,)
Train set after Amputation (282432, 5)
X_train Amputed value count of varaible  84730
Test set after Amputation (70608, 5)
X_test Amputed value count of varaible  21182
Simple Imputation using sampling
['gender', 'usertype']
Now running model is .... birthyear
Train (282432, 5) (282432,)
Test (7

In [35]:
bootstrap_data_sample

{'gender': {'R2': [0.16919905790204676, 0.1826797666015309],
  'RMSE': [650.2069516025169, 682.5137713422105],
  'intercept': [4128.493524077371, 5376.421720617557]},
 'dist': {'R2': [0.15761101587638798, 0.17022436215906783],
  'RMSE': [655.4337064017885, 687.3741034288978],
  'intercept': [3028.466203983856, 4121.852654760087]},
 'hour': {'R2': [0.16968842201072198, 0.18295193666771514],
  'RMSE': [650.0568054301013, 682.372299749865],
  'intercept': [3677.489630740998, 4810.190489413755]},
 'birthyear': {'R2': [0.16996440700846982, 0.1834617553890609],
  'RMSE': [649.8478843106484, 682.2068316379654],
  'intercept': [2498.366344798944, 3716.5799342012883]},
 'usertype': {'R2': [0.11950140076877447, 0.13177373899652223],
  'RMSE': [670.4128514173087, 703.1189583773973],
  'intercept': [2380.6799265021723, 3559.308188143546]}}